# 二、解决 Bad Case

在本节中，我们结合项目实际，讲解找出并针对性优化 Prompt 以解决 Bad Case 的一般思路。

## 1. 构造向量数据库

首先，我们根据上一章的内容，加载向量数据库及检索链：

In [2]:
from langchain.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings    # 调用 OpenAI 的 Embeddings 模型
import openai
from dotenv import load_dotenv, find_dotenv
import os

_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

# 定义 Embeddings
embedding = OpenAIEmbeddings() 

# 向量数据库持久化路径
persist_directory = '../../data_base/vector_db/chroma'

# 加载数据库
vectordb = Chroma(
    persist_directory=persist_directory,  # 允许我们将persist_directory目录保存到磁盘上
    embedding_function=embedding
)

print(f"向量库中存储的数量：{vectordb._collection.count()}")

Using embedded DuckDB with persistence: data will be stored in: ../../data_base/vector_db/chroma


向量库中存储的数量：1120


接着我们先使用初始化的 Prompt 创建一个基于模板的检索链：

In [103]:
from langchain.prompts import PromptTemplate

template_v1 = """使用以下上下文来回答最后的问题。如果你不知道答案，就说你不知道，不要试图编造答
案。最多使用三句话。尽量使答案简明扼要。总是在回答的最后说“谢谢你的提问！”。
{context}
问题: {question}
有用的回答:"""

QA_CHAIN_PROMPT = PromptTemplate(input_variables=["context","question"],
                                 template=template_v1)

from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(model_name = "gpt-3.5-turbo", temperature = 0 )

qa_chain = RetrievalQA.from_chain_type(llm,
                                       retriever=vectordb.as_retriever(),
                                       return_source_documents=True,
                                       chain_type_kwargs={"prompt":QA_CHAIN_PROMPT})



测试一下效果：

In [9]:
question = "什么是南瓜书"
result = qa_chain({"query": question})
print(result["result"])

南瓜书是对《机器学习》（西瓜书）中难以理解的公式进行解析和补充推导细节的一本书。谢谢你的提问！


可以查看检索到的相关文本：

In [10]:
result

{'query': '什么是南瓜书',
 'result': '南瓜书是对《机器学习》（西瓜书）中难以理解的公式进行解析和补充推导细节的一本书。谢谢你的提问！',
 'source_documents': [Document(page_content='下学生”。\n使用说明\n• 南瓜书的所有内容都是以西瓜书的内容为前置知识进行表述的，所以南瓜书的最佳使用方法是以西瓜书\n为主线，遇到自己推导不出来或者看不懂的公式时再来查阅南瓜书；\n• 对于初学机器学习的小白，西瓜书第 1 章和第 2 章的公式强烈不建议深究，简单过一下即可，等你学得\n有点飘的时候再回来啃都来得及；\n• 每个公式的解析和推导我们都力 (zhi) 争 (neng) 以本科数学基础的视角进行讲解，所以超纲的数学知识\n我们通常都会以附录和参考文献的形式给出，感兴趣的同学可以继续沿着我们给的资料进行深入学习；\n• 若南瓜书里没有你想要查阅的公式，或者你发现南瓜书哪个地方有错误，请毫不犹豫地去我们 GitHub 的\nIssues（地址：https://github.com/datawhalechina/pumpkin-book/issues）进行反馈，在对应版块\n提交你希望补充的公式编号或者勘误信息，我们通常会在 24 小时以内给您回复，超过 24 小时未回复的\n话可以微信联系我们（微信号：at-Sm1les）；', metadata={'source': '../../data_base/knowledge_db/pumkin_book/pumpkin_book.pdf', 'file_path': '../../data_base/knowledge_db/pumkin_book/pumpkin_book.pdf', 'page': 1, 'total_pages': 196, 'format': 'PDF 1.5', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'creator': 'LaTeX with hyperref', 'producer': 'xdvipdfmx (20200315)', 'creationDate': "D:20230303170709-00'00'", 'modDate': '', '

## 2. 提升直观回答质量

寻找 Bad Case 的思路有很多，最直观也最简单的就是评估直观回答的质量，结合原有资料内容，判断在什么方面有所不足。例如，上述的测试我们可以构造成一个 Bad Case：

    问题：什么是南瓜书
    初始回答：南瓜书是对《机器学习》（西瓜书）中难以理解的公式进行解析和补充推导细节的一本书。谢谢你的提问！
    存在不足：回答太简略，需要回答更具体；谢谢你的提问感觉比较死板，可以去掉
我们再针对性地修改 Prompt 模板，加入要求其回答具体，并去掉“谢谢你的提问”的部分：

In [4]:
template_v2 = """使用以下上下文来回答最后的问题。如果你不知道答案，就说你不知道，不要试图编造答
案。你应该使答案尽可能详细具体，但不要偏题。如果答案比较长，请酌情进行分段，以提高答案的阅读体验。
{context}
问题: {question}
有用的回答:"""

QA_CHAIN_PROMPT = PromptTemplate(input_variables=["context","question"],
                                 template=template_v2)
qa_chain = RetrievalQA.from_chain_type(llm,
                                       retriever=vectordb.as_retriever(),
                                       return_source_documents=True,
                                       chain_type_kwargs={"prompt":QA_CHAIN_PROMPT})

question = "什么是南瓜书"
result = qa_chain({"query": question})
print(result["result"])

南瓜书是一本机器学习领域的辅助教材，旨在对《机器学习》（西瓜书）中比较难理解的公式进行解析和补充推导细节。它是以西瓜书的内容为基础，对部分公式的推导细节进行详细讲解，帮助读者更好地理解和掌握机器学习的基本原理和方法。南瓜书的使用方法是在阅读西瓜书时遇到自己推导不出来或者看不懂的公式时再来查阅，以帮助读者更好地理解和应用机器学习算法。南瓜书的目标读者是初学机器学习的人群，尤其是那些对数学基础较弱的读者，通过南瓜书的解析和补充，希望能够帮助他们更轻松地理解和学习机器学习的知识。


可以看到，改进后的 v2 版本能够给出更具体、详细的回答，解决了之前的问题。但是我们可以进一步思考，要求模型给出具体、详细的回答，是否会导致针对一些有要点的回答没有重点、模糊不清？我们测试以下问题：

In [15]:
question = "使用大模型时，构造 Prompt 的原则有哪些"
result = qa_chain({"query": question})
print(result["result"])

构造 Prompt 的原则有两个关键点：编写清晰、具体的指令和给予模型充足思考时间。

首先，Prompt 需要清晰明确地表达需求，提供充足上下文，使语言模型准确理解我们的意图。这就要求我们以清晰、详尽的语言表达 Prompt，避免歧义和模糊性。在设计 Prompt 时，可以使用更长、更复杂的语句，以提供更丰富的上下文和细节，帮助模型更准确地把握所需的操作和响应方式。

其次，让语言模型有充足时间推理也非常重要。语言模型需要时间来思考并解决复杂问题，就像人类解题一样。如果让语言模型匆忙给出结论，其结果很可能不准确。因此，在设计 Prompt 时，可以要求模型先列出对问题的各种看法，说明推理依据，然后再得出最终结论。这样的逐步推理要求能够让语言模型投入更多时间逻辑思维，输出结果也将更可靠准确。

通过遵循这两个原则，我们可以优化 Prompt，使语言模型能够尽可能发挥潜力，完成复杂的推理和生成任务。这些原则是开发者取得语言模型应用成功的重要一步。在设计 Prompt 时，我们需要不断尝试和优化，通过多次迭代来逼近最优的 Prompt 形式。这需要智慧和毅力，但结果往往是值得的。

总之，构造 Prompt 的原则是编写清晰、具体的指令和给予模型充足思考时间。这样可以帮助语言模型准确理解需求，并提高处理复杂问题的效果。


可以看到，针对我们关于 LLM 课程的提问，模型回答确实详细具体，也充分参考了课程内容，但回答使用首先、其次等词开头，同时将整体答案分成了4段，导致答案不是特别重点清晰，不容易阅读。因此，我们构造以下 Bad Case：

    问题：使用大模型时，构造 Prompt 的原则有哪些
    初始回答：略
    存在不足：没有重点，模糊不清

针对该 Bad Case，我们可以改进 Prompt，要求其对有几点的答案进行分点标号，让答案清晰具体：

In [18]:
template_v3 = """使用以下上下文来回答最后的问题。如果你不知道答案，就说你不知道，不要试图编造答
案。你应该使答案尽可能详细具体，但不要偏题。如果答案比较长，请酌情进行分段，以提高答案的阅读体验。
如果答案有几点，你应该分点标号回答，让答案清晰具体
{context}
问题: {question}
有用的回答:"""

QA_CHAIN_PROMPT = PromptTemplate(input_variables=["context","question"],
                                 template=template_v3)
qa_chain = RetrievalQA.from_chain_type(llm,
                                       retriever=vectordb.as_retriever(),
                                       return_source_documents=True,
                                       chain_type_kwargs={"prompt":QA_CHAIN_PROMPT})

question = "使用大模型时，构造 Prompt 的原则有哪些"
result = qa_chain({"query": question})
print(result["result"])

构造 Prompt 的原则有两个关键点：

1. 编写清晰、具体的指令：Prompt 需要以清晰、具体的方式表达需求，并提供充足的上下文，使语言模型准确理解意图。过于简略的 Prompt 往往使模型难以把握具体任务。因此，在设计 Prompt 时，需要用清晰、详尽的语言表达需求，给出足够的上下文和细节，让模型可以更准确地理解操作和响应方式。

2. 给予模型充足思考时间：语言模型与人类一样，需要时间来思考并解决复杂问题。如果让语言模型匆忙给出结论，其结果很可能不准确。因此，在设计 Prompt 时，需要给予语言模型充足的推理时间。可以要求模型先列出对问题的各种看法，说明推理依据，然后再得出最终结论。通过添加逐步推理的要求，可以让语言模型投入更多时间逻辑思维，输出结果也将更可靠准确。

这两个原则的应用可以帮助语言模型尽可能发挥潜力，完成复杂的推理和生成任务。在设计 Prompt 时，需要注意清晰明确地表达需求，并给予模型充足的思考时间，以提高语言模型的性能和可靠性。


提升回答质量的方法还有很多，核心是围绕具体业务展开思考，找出初始回答中不足以让人满意的点，并针对性进行提升改进，此处不再赘述。

## 3. 标明知识来源，提高可信度

由于大模型存在幻觉问题，有时我们会怀疑模型回答并非源于已有知识库内容，这对一些需要保证真实性的场景来说尤为重要，例如：

In [15]:
question = "强化学习的定义是什么"
result = qa_chain({"query": question})
print(result["result"])

强化学习的定义是一种机器学习方法，用于让智能体在与环境的交互中学习如何做出一系列好的决策。在强化学习中，智能体通过观察环境的状态，选择一个动作来执行，并根据环境的反馈（奖励或惩罚）来调整自己的决策策略，以最大化长期累积的奖励。强化学习的目标是在不确定的情况下，通过与环境的交互来学习最优的决策策略。


In [17]:
template_v4 = """使用以下上下文来回答最后的问题。如果你不知道答案，就说你不知道，不要试图编造答
案。你应该使答案尽可能详细具体，但不要偏题。如果答案比较长，请酌情进行分段，以提高答案的阅读体验。
如果答案有几点，你应该分点标号回答，让答案清晰具体。
请你附上回答的来源原文，以保证回答的正确性。
{context}
问题: {question}
有用的回答:"""

QA_CHAIN_PROMPT = PromptTemplate(input_variables=["context","question"],
                                 template=template_v4)
qa_chain = RetrievalQA.from_chain_type(llm,
                                       retriever=vectordb.as_retriever(),
                                       return_source_documents=True,
                                       chain_type_kwargs={"prompt":QA_CHAIN_PROMPT})

question = "强化学习的定义是什么"
result = qa_chain({"query": question})
print(result["result"])

强化学习的定义是一种机器学习方法，用于学习如何在不确定的环境中做出一系列决策，以最大化累积奖励。在强化学习中，智能体与环境进行交互，智能体根据当前状态选择动作，环境根据智能体的动作给予奖励或惩罚，并转移到下一个状态。智能体通过不断尝试和学习，逐渐优化决策策略，以获得更高的累积奖励。强化学习的应用广泛，包括游戏玩法、机器人控制、交通优化等领域。来源：蘑菇书一语二语二强化学习教程


但是，附上原文来源往往会导致上下文的增加以及回复速度的降低，我们需要根据业务场景酌情考虑是否要求附上原文。

## 4. 构造思维链

大模型往往可以很好地理解并执行指令，但模型本身还存在一些能力的限制，例如大模型的幻觉、无法理解较为复杂的指令、无法执行复杂步骤等。我们可以通过构造思维链，将 Prompt 构造成一系列步骤来尽量减少其能力限制，例如，我们可以构造一个两步的思维链，要求模型在第二步做出反思，以尽可能消除大模型的幻觉问题。

我们首先有这样一个 Bad Case：

    问题：我们应该如何去构造一个 LLM 项目
    初始回答：略
    存在不足：事实上，知识库中并没有关于如何构造LLM项目的内容，模型的回答看似有道理，实则是大模型的幻觉，将部分相关的文本拼接得到，存在问题

In [35]:
question = "我们应该如何去构造一个LLM项目"
result = qa_chain({"query": question})
print(result["result"])

构造一个 LLM 项目需要以下步骤：

1. 确定项目目标：首先，你需要明确你的项目目标是什么。你想要使用 LLM 来完成什么任务？例如，你可能想要构建一个文本生成模型，用于自动写作；或者你可能想要构建一个文本分类模型，用于情感分析。明确项目目标对于后续的步骤非常重要。

2. 收集和准备数据：接下来，你需要收集和准备用于训练 LLM 的数据。这包括找到合适的数据源，并将数据进行清洗和预处理。确保你的数据集具有足够的多样性和代表性，以便训练出具有良好泛化能力的模型。

3. 设计 Prompt：Prompt 是你向 LLM 提供的指令或问题。设计一个好的 Prompt 对于获得准确和有用的结果非常重要。你可以根据你的项目目标和需求来设计 Prompt，确保它能够引导 LLM 生成符合你期望的文本。

4. 微调模型：在设计好 Prompt 后，你需要使用训练数据对 LLM 进行微调。微调是指在预训练的基础模型上进一步训练，使其适应特定任务或领域。通过微调，你可以提高模型在特定任务上的性能和效果。

5. 评估和优化：在微调完成后，你需要评估模型的性能并进行优化。使用一些评估指标来衡量模型的准确性、流畅性和一致性。如果模型表现不佳，你可以尝试调整 Prompt 的设计或增加训练数据来改进模型。

6. 部署和应用：最后，当你对模型的性能满意后，你可以将其部署到实际应用中。根据你的需求，你可以选择将模型封装成 API，以便其他应用程序可以调用；或者你可以将模型集成到自己的应用程序中，以实现特定的功能。

总结：
构造一个 LLM 项目需要明确项目目标、收集和准备数据、设计 Prompt、微调模型、评估和优化以及部署和应用。每个步骤都需要仔细考虑和执行，以确保项目的成功和效果。


对此，我们可以优化 Prompt，将之前的 Prompt 变成两个步骤，要求模型在第二个步骤中做出反思：

In [104]:
template_v4 = """
请你依次执行以下步骤：
① 使用以下上下文来回答最后的问题。如果你不知道答案，就说你不知道，不要试图编造答案。
你应该使答案尽可能详细具体，但不要偏题。如果答案比较长，请酌情进行分段，以提高答案的阅读体验。
如果答案有几点，你应该分点标号回答，让答案清晰具体。
上下文：
{context}
问题: 
{question}
有用的回答:
② 基于提供的上下文，反思回答中有没有不正确或不是基于上下文得到的内容，如果有，修改回答"""

QA_CHAIN_PROMPT = PromptTemplate(input_variables=["context","question"],
                                 template=template_v4)
qa_chain = RetrievalQA.from_chain_type(llm,
                                       retriever=vectordb.as_retriever(),
                                       return_source_documents=True,
                                       chain_type_kwargs={"prompt":QA_CHAIN_PROMPT})

question = "我们应该如何去构造一个LLM项目"
result = qa_chain({"query": question})
print(result["result"])

根据上下文，我们无法得知如何构造一个LLM项目。上下文中提到了LLM的一些应用场景和功能，但没有提供关于如何构造LLM项目的具体信息。因此，我们无法回答这个问题。


可以看出，要求模型做出自我反思之后，模型修复了自己的幻觉，告诉我们知识库中并没有提及如何构造一个 LLM 项目。我们还可以通过构造思维链完成更多功能，此处就不再赘述了，欢迎读者尝试。

## 5. 增加一个指令解析

我们往往会面临一个需求，即我们需要模型以我们指定的格式进行输出。但是，由于我们使用了 Prompt Template 来填充用户问题，用户问题中存在的格式要求往往会被忽略，例如：

In [105]:
question = "LLM的分类是什么？给我返回一个 Python List"
result = qa_chain({"query": question})
print(result["result"])

根据提供的上下文，LLM的分类可以分为基础LLM和指令微调LLM。


可以看到，虽然我们要求模型给返回一个 Python List，但该输出要求被包裹在 Template 中被模型忽略掉了。针对该问题，我们可以构造一个 Bad Case：

    问题：LLM的分类是什么？给我返回一个 Python List
    初始回答：根据提供的上下文，LLM的分类可以分为基础LLM和指令微调LLM。
    存在不足：没有按照指令中的要求输出

针对该问题，一个存在的解决方案是，在我们的检索 LLM 之前，增加一层 LLM 来实现指令的解析，将用户问题的格式要求和问题内容拆分开来。这样的思路其实就是目前大火的 Agent 机制的雏形，即针对用户指令，设置一个 LLM（即 Agent）来理解指令，判断指令需要执行什么工具，再针对性调用需要执行的工具，其中每一个工具可以是基于不同 Prompt Engineering 的 LLM，也可以是例如数据库、API 等。LangChain 中其实有设计 Agent 机制，但本教程中我们就不再赘述了，这里只基于 OpenAI 的原生接口简单实现这一功能：

In [106]:
# 定义一个 OpenAI 的原生接口
import openai
# 一个封装 OpenAI 接口的函数，参数为 Prompt，返回对应结果
def get_completion(prompt, model="gpt-3.5-turbo", temperature = 0):
    '''
    prompt: 对应的提示词
    model: 调用的模型，默认为 gpt-3.5-turbo(ChatGPT)，有内测资格的用户可以选择 gpt-4
    '''
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature, # 模型输出的温度系数，控制输出的随机程度
    )
    # 调用 OpenAI 的 ChatCompletion 接口
    return response.choices[0].message["content"]

prompt_input = '''
请判断以下问题中是否包含对输出的格式要求，并按以下要求输出：
请返回给我一个可解析的Python列表，列表第一个元素是对输出的格式要求，应该是一个指令；第二个元素是去掉格式要求的问题原文
如果没有格式要求，请将第一个元素置为空
需要判断的问题：
```
{}
```
'''



我们测试一下该 LLM 分解格式要求的能力：

In [107]:
response = get_completion(prompt_input.format(question))
response

"['给我返回一个 Python List', 'LLM的分类是什么？']"

可以看到，通过上述 Prompt，LLM 可以很好地实现输出格式的解析，接下来，我们可以再设置一个 LLM 根据输出格式要求，对输出内容进行解析：

In [108]:
prompt_output = '''
请根据回答文本和输出格式要求，按照给定的格式要求对问题做出回答
需要回答的问题：
```
{}
```
回答文本：
```
{}
```
输出格式要求：
```
{}
```
'''

然后我们可以将两个 LLM 与检索链串联起来：


In [109]:
question = 'LLM的分类是什么？给我返回一个 Python List'
# 首先将格式要求与问题拆分
input_lst_s = get_completion(prompt_input.format(question))
rule, new_question = eval(input_lst_s)
# 接着使用拆分后的问题调用检索链
result = qa_chain({"query": new_question})
result_context = result["result"]
# 接着调用输出格式解析
response = get_completion(prompt_output.format(new_question, result_context, rule))
response

"['基础LLM', '指令微调LLM']"

可以看到，经过如上步骤，我们就成功地实现了输出格式的限定。当然，在上面代码中，核心为介绍 Agent 思想，事实上，不管是 Agent 机制还是 Parser 机制（也就是限定输出格式），LangChain 都提供了成熟的工具链供使用，欢迎感兴趣的读者深入探讨，此处就不展开讲解了。

通过上述讲解的思路，结合实际业务情况，我们可以不断发现 Bad Case 并针对性优化 Prompt，但是为了保证系统的稳定性避免负优化，我们需要将所有的 Bad Case 维护一个验证集合，来验证每一次优化后系统在整个验证集上的效果，以判断我们是否成功做出了优化。那么，如何评估以大模型为核心的系统的性能呢？接下来我们将介绍大模型的一般评估方法。